In [ ]:
import os

# generate and download api key file from https://portal.variantperception.com/vp-data-api
# save the value in your local user directory (C:/Users/John/vp_api_key.txt)
# or set the key as an environment variable: VP_ANALYSIS_API_KEY 
if "VP_ANALYSIS_API_KEY" in os.environ:
    print("Reading API key from environment")
    api_key = os.environ.get("VP_ANALYSIS_API_KEY")
else:
    with open(os.path.expanduser('~/vp_api_key.txt'), 'r') as file:
        print("Reading API key from file")
        api_key = file.read().replace('\n', '')
print(f"API Key: {api_key[:5]}...")
api = VPAnalysisAPI(api_key)

Reading API key from environment
API Key: eyJhb...


In [2]:
# There are two data types: 
# 1. macro time series 
# 2. time series for a particular asset and factor

# First, we show how macro data works
# This will return a list of all macro data identifiers
macro_series = api.get_macro_series_list()
macro_series

,ticker,name,model,asset,region
0,bcfi24,VP BCFI,liquidity,macro,world
1,cb_policy_au_hiking,AU Central Bank Hiking Regime,cb_policy,macro,au
2,cb_policy_br_hiking,BR Central Bank Hiking Regime,cb_policy,macro,br
3,cb_policy_ca_hiking,CA Central Bank Hiking Regime,cb_policy,macro,ca
4,cb_policy_ch_hiking,CH Central Bank Hiking Regime,cb_policy,macro,ch
...,...,...,...,...,...
1306,vp_macro_regime_prob,VP Macro Risk Indicator,macro_regimes,macro,world
1307,vp_macro_risk_gr_prob,VP Macro Risk Indicator - Growth Dimension,macro_regimes,macro,world
1308,vp_macro_risk_infl_prob,VP Macro Risk Indicator - Inflation Dimension,macro_regimes,macro,world
1309,vp_macro_risk_liq_prob,VP Macro Risk Indicator - Liquidity Dimension,macro_regimes,macro,world


In [ ]:
# Search the list for the desired ticker (or use the interactive UI on the portal)
lei_us_grow_id = macro_series.query("name == 'United States Real GDP Leading Indicator'").ticker.squeeze()
print(lei_us_grow_id)

ex_liq_id = macro_series.query("name == 'Global Excess Liquidity'").ticker.squeeze()
print(ex_liq_id)

# Fetch the data
# The column headers will show VP's data id format, which is vp:ticker, with [0] at the end for first revision data
df = api.get_df_from_macro_series_list([lei_us_grow_id, ex_liq_id], start_date='2010-01-01', end_date='2011-01-01', first_revision=True)
df

lei_us_grow_rgdp
globexliq24


,vp:lei_us_grow_rgdp[0],vp:globexliq24[0]
dt,,
2010-01-31,2.060704,10.728616
2010-02-28,2.390395,9.098519
2010-03-31,2.336014,10.799719
2010-04-30,2.740704,5.755906
2010-05-31,3.526895,2.575121
2010-06-30,2.998410,-5.845586
2010-07-31,2.055410,-4.001016
2010-08-31,1.808110,-1.544045
2010-09-30,1.665542,-2.004566


In [ ]:
# To get last revision data:
df = api.get_df_from_macro_series_list([lei_us_grow_id, ex_liq_id], start_date='2010-01-01', end_date='2011-01-01', first_revision=False)
df

,vp:lei_us_grow_rgdp,vp:globexliq24
dt,,
2010-01-31,4.835555,11.009721
2010-02-28,5.193266,9.605885
2010-03-31,5.012432,11.360102
2010-04-30,4.937665,6.420208
2010-05-31,5.595013,3.039884
2010-06-30,5.157148,-5.326997
2010-07-31,4.279086,-3.440219
2010-08-31,4.112647,-1.174892
2010-09-30,4.207311,-1.556803


In [ ]:
# Next, there is asset-factor data

# This will return all the public factors:
factors = api.get_factors()
factors

,factor_identifier,factor_name,factor_description
14,capital_cycle_score,Capital Cycle Score,Capital Cycle Score
21,ccl_invs,Capital Cycle Inputs: Net Investment,Capital Cycle Inputs: Net Investment
22,ccl_rmw,Capital Cycle Inputs: Net ROIC,Capital Cycle Inputs: Net ROIC
23,ccl_rxs,Capital Cycle Inputs: ROIC Ranking,Capital Cycle Inputs: ROIC Ranking
50,fval,Fair Value Upside,Percentage upside/downside relative to VP Fair...
54,fxe_sc,FX Edge (Scaled),FX Edge (Scaled)
70,lp_brd,LPPL Breadth,Percentage of LPPL net buy (buy minus sell) si...
73,lppl_fnd,LPPL Pattern Found,"LPPL Pattern Found: +1 for buys, -1 for sells"
74,lppl_sig,LPPL Signal Trigger,"LPPL Signal Trigger: +1 for buys, -1 for sells"
110,quality_score,Quality,VP Quality Score


In [ ]:
# This returns all the macro assets:
macro = api.get_macro_assets()
macro

,asset_id,name,assetclass,country
0,spx,S&P 500 Index (ETF: SPY),equity region,us
1,qqq,Nasdaq 100 ETF (QQQ),equity region,us
2,iwm,Russell 2000 ETF (IWM),equity region,us
3,sx5e,Euro STOXX 50,equity region,eu
4,cac,France CAC 40,equity region,fr
...,...,...,...,...
311,ushysp,USD High Yield Spread,fixed income,us
312,usigsp,USD Investment Grade Spread,fixed income,us
313,euhysp,Euro High Yield Spread,fixed income,us
314,euigsp,Euro Investment Grade Spread,fixed income,us


In [ ]:
# This returns the single stock equity assets:
equities = api.get_equity_assets()
equities

,asset_id,name,country,industry
0,000001:SZSE,"Ping An Bank Co., Ltd.",China,Banks
1,000002:SZSE,"China Vanke Co., Ltd.",China,Real Estate Management and Development
2,000005:SZSE:20240305,Shenzhen Fountain Corporation,China,Construction and Engineering
3,000006:SZSE,"Shenzhen Zhenye (Group) Co.,Ltd.",China,Real Estate Management and Development
4,000008:SZSE,"China High-Speed Railway Technology Co., Ltd.",China,Ground Transportation
...,...,...,...,...
28514,ZYW:MUN:20231016,China Properties Group Limited,Hong Kong,Real Estate Management and Development
28515,ZYXI.Q:OTCPK,"Zynex, Inc.",United States,Health Care Equipment and Supplies
28516,ZZ B:OM,Zinzino AB (publ),Sweden,Distributors
28517,ZZ:NYSE:20130315,Sealy Corporation,United States,Household Durables


In [ ]:
# To get asset-factor data, find the asset id and factor id of the data you are looking for
# You can do this by searching through the dataframes above, or by using the interactive UI on the portal
fast_money_id = factors.query("factor_name == 'VP Fast Money'").factor_identifier.squeeze()
print(fast_money_id)
cap_cycle_id = factors.query("factor_name == 'Capital Cycle Score'").factor_identifier.squeeze()
print(cap_cycle_id)

spx_id = macro.query("name == 'S&P 500 Index (ETF: SPY)'").asset_id.squeeze()
print(spx_id)
apple_id = equities.query("name == 'Apple Inc.'").asset_id.squeeze()
print(apple_id)

# Finally, fetch the data
df = api.get_df_from_asset_factor_list([(spx_id, fast_money_id), (apple_id, cap_cycle_id)], start_date='2010-01-01', end_date='2010-01-15', first_revision=True)

# The column headers will contain VP's data id format, which is vp:ss:asset$factor (and [0] for first revision)
df

vp_fast_money
capital_cycle_score
spx
AAPL:NasdaqGS


,vp:ss:AAPL:NasdaqGS$capital_cycle_score[0],vp:ss:spx$vp_fast_money[0]
dt,,
2010-01-01,76.749874,57.164359
2010-01-02,NaN,NaN
2010-01-03,NaN,NaN
2010-01-04,72.813975,58.463381
2010-01-05,72.832861,58.446056
2010-01-06,72.851747,58.352957
2010-01-07,72.870633,58.533319
2010-01-08,72.889518,58.759974
2010-01-09,NaN,NaN
